# Chapter 10. Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. 
![Split Apply Combine](images/split-apply-combine.png)

In [1]:
import pandas as pd
import numpy as np

%config Completer.use_jedi = False

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : [1, 2, 3, 4, 5],
                  'data2' : [10, 20, 30, 40, 50]})

df

,key1,key2,data1,data2
0,a,one,1,10
1,a,two,2,20
2,b,one,3,30
3,b,two,4,40
4,a,one,5,50


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# the name of the index is 'key1'
grouped.mean()

key1
a    2.666667
b    3.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     3
      two     2
b     one     3
      two     4
Name: data1, dtype: int64

In [6]:
means.unstack()

key2,one,two
key1,,
a,3,2
b,3,4


In [7]:
# You can use arrays from outside to group data
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    2.0
            2006    3.0
Ohio        2005    2.5
            2006    5.0
Name: data1, dtype: float64

In [8]:
# non numerical data 'key2' is hidden (nuisance column)
df.groupby('key1').mean()

,data1,data2
key1,,
a,2.666667,26.666667
b,3.500000,35.000000


In [9]:
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one       3     30
     two       2     20
b    one       3     30
     two       4     40

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2  data1  data2
0    a  one      1     10
1    a  two      2     20
4    a  one      5     50
b
  key1 key2  data1  data2
2    b  one      3     30
3    b  two      4     40


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      1     10
4    a  one      5     50
('a', 'two')
  key1 key2  data1  data2
1    a  two      2     20
('b', 'one')
  key1 key2  data1  data2
2    b  one      3     30
('b', 'two')
  key1 key2  data1  data2
3    b  two      4     40


In [13]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2  data1  data2
 0    a  one      1     10
 1    a  two      2     20
 4    a  one      5     50,
 'b':   key1 key2  data1  data2
 2    b  one      3     30
 3    b  two      4     40}

In [14]:
df.dtypes

key1     object
key2     object
data1     int64
data2     int64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

int64
   data1  data2
0      1     10
1      2     20
2      3     30
3      4     40
4      5     50
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
The code above is syntactic sugar for:
```
df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key2'])
```

### Grouping with Dicts and Series

In [16]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

people

,a,b,c,d,e
Joe,0.008127,-0.794072,0.220529,-0.008018,-0.566512
Steve,0.977444,-1.137005,-0.382100,-0.138602,-1.212685
Wes,1.553380,NaN,NaN,-0.689204,-0.648550
Jim,-0.396258,0.314197,-0.723778,0.767941,-0.136225
Travis,-0.667937,0.651969,1.197329,0.119727,-0.833586


In [17]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [18]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.212510,-1.352456
Steve,-0.520701,-1.372246
Wes,-0.689204,0.904830
Jim,0.044164,-0.218286
Travis,1.317056,-0.849554


In [19]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [20]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with functions

In [21]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.165250,-0.479875,-0.503249,0.070719,-1.351287
5,0.977444,-1.137005,-0.382100,-0.138602,-1.212685
6,-0.667937,0.651969,1.197329,0.119727,-0.833586


In [22]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.008127 -0.794072  0.220529 -0.689204 -0.648550
  two -0.396258  0.314197 -0.723778  0.767941 -0.136225
5 one  0.977444 -1.137005 -0.382100 -0.138602 -1.212685
6 two -0.667937  0.651969  1.197329  0.119727 -0.833586

### Grouping by Index Levels

In [23]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.660688  0.698512  0.440712 -0.274461  0.044336
1      0.820930 -0.729110  0.068338 -0.580388  0.583933
2     -1.790123  0.817868  0.382642 -0.843192 -0.653588
3      0.900733  0.185432  0.342786 -0.086842  1.418865

In [24]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

Aggregation: any data transformation that produces scalar values from arrays.

In [25]:
df

,key1,key2,data1,data2
0,a,one,1,10
1,a,two,2,20
2,b,one,3,30
3,b,two,4,40
4,a,one,5,50


In [26]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    4.4
b    3.9
Name: data1, dtype: float64

In [27]:
# custom function is much slower than the optimized builtin functions
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,40
b,1,10


In [28]:
grouped.describe()

data1                                                data2             \
     count      mean       std  min   25%  50%   75%  max count       mean   
key1                                                                         
a      3.0  2.666667  2.081666  1.0  1.50  2.0  3.50  5.0   3.0  26.666667   
b      2.0  3.500000  0.707107  3.0  3.25  3.5  3.75  4.0   2.0  35.000000   

                                               
            std   min   25%   50%   75%   max  
key1                                           
a     20.816660  10.0  15.0  20.0  35.0  50.0  
b      7.071068  30.0  32.5  35.0  37.5  40.0

### Column-Wise and Multiple Function Application

In [30]:
tips = pd.read_csv('pydata-book/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [31]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [32]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [33]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [35]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [36]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [37]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [38]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [39]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [40]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine